In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
file = 'final.csv'
df = pd.read_csv(file)
df = pd.DataFrame(df)
df

,Год,Направление,Уровень безработицы,Уровень инфляции,Вступительный балл,Пол,Образовательные специальные потребности,Дневная/вечерняя форма обучения,Предыдущее образование (оценка),Перемещенный,Возраст при зачислении,Международный статус,ВВП,Количество абитуриентов
0,2022.0,0.0,10.8,1.4,127.3,1,0,1,122.0,1,20,0,1.74,215
1,2023.0,1.0,13.9,-0.3,142.5,1,0,1,160.0,1,19,0,0.79,252
2,2024.0,2.0,10.8,1.4,124.8,1,0,1,122.0,1,19,0,1.74,226
3,2022.0,3.0,9.4,-0.8,119.6,0,0,1,122.0,1,20,0,-3.12,331
4,2023.0,4.0,13.9,-0.3,141.5,0,0,0,100.0,0,45,0,0.79,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,2022.0,3.0,15.5,2.8,122.2,1,0,1,125.0,0,19,0,-4.06,331
4420,2023.0,3.0,11.1,0.6,119.0,0,0,1,120.0,1,18,1,2.02,331
4421,2024.0,6.0,13.9,-0.3,149.5,0,0,1,154.0,1,30,0,0.79,766
4422,2022.0,13.0,9.4,-0.8,153.8,0,0,1,180.0,1,20,0,-3.12,380


In [42]:
from sklearn.model_selection import train_test_split, cross_val_score

X = df[['Пол', 'Возраст при зачислении', 'Дневная/вечерняя форма обучения', 
        'Вступительный балл', 'Направление','Предыдущее образование (оценка)']]
y = df['Количество абитуриентов']

# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)

for i in range(n_iter):
    # Случайное разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

accuracies_log = []  # Список для хранения точности на каждой итерации

# Создание и обучение модели (Логистическая регрессия)
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Предсказание
y_pred_logistic = logistic_model.predict(X_test)

# Оценка точности модели
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracies_log.append(accuracy_logistic)
print(f'Итерация {i + 1}: Точность модели = {accuracy_logistic:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_log)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 10: Точность модели = 0.76
Средняя точность за 10 итераций: 0.76


c:\Users\1645295\Documents\GitHub\diplom\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
from sklearn.ensemble import RandomForestClassifier

accuracies_for = []  # Список для хранения точности на каждой итерации

# Создание и обучение модели (Random Forest)
forest_model = RandomForestClassifier(random_state=42)
forest_model.fit(X_train, y_train)
    
# Предсказание
y_pred_for = forest_model.predict(X_test)
    
# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred_for)
accuracies_for.append(accuracy)
print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_for)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 10: Точность модели = 0.98
Средняя точность за 10 итераций: 0.98


In [45]:
from sklearn.tree import DecisionTreeClassifier

accuracies_tree = []  # Список для хранения точности на каждой итерации

# Создание и обучение модели
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Предсказание 
y_pred_tree = decision_tree_model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred_tree)
accuracies_tree.append(accuracy)
print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_tree)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 10: Точность модели = 1.00
Средняя точность за 10 итераций: 1.00


In [46]:
from xgboost import XGBClassifier

accuracies_xgb = []  # Список для хранения точности на каждой итерации

# Создание и обучение модели
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Предсказание
y_pred_xgb = xgb_model.predict(X_test)


# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred_xgb)
accuracies_xgb.append(accuracy)
print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_xgb)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], got [ 12  86 141 170 192 210 215 226 252 268 331 337 355 380 766]